# AirBnB NY Locations Data Case Study

In this final project, you task will be to take the data provided and find evidance to answer the following questions.

1. Which hosts are the busiest and why?
2. How many neighborhood groups are available and which shows up the most?
3. Are private rooms the most popular in manhattan?
4. Which hosts are the busiest and based on their reviews?
5. Which neighorhood group has the highest average price?
6. Which neighborhood group has the highest total price?
7. Which top 5 hosts have the highest total price?
8. Who currently has no (zero) availability with a review count of 100 or more?
9. What host has the highest total of prices and where are they located?
10. When did Danielle from Queens last receive a review?


In [12]:
import pandas as pd
import numpy as np

ab_data= pd.read_csv(r'C:\Users\Logan\Documents\GitHub\bootcamp\week_3\AB_NYC_2019.csv')
print(ab_data.keys())
display(ab_data['id'].duplicated) # 
'''just making sure no property ID's are duplicated.
I definitely suspect that host_id is duplicated since one host
can have many properties.'''

ab_data.head()

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')


<bound method Series.duplicated of 0            2539
1            2595
2            3647
3            3831
4            5022
           ...   
48890    36484665
48891    36485057
48892    36485431
48893    36485609
48894    36487245
Name: id, Length: 48895, dtype: int64>

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [13]:
#which hosts are the busiest and why

#I have a feeling popularity will be need to accessed by other metrics in the future
ab_sum= ab_data.groupby(['host_id','host_name'], as_index=False).sum(True).sort_values('reviews_per_month', ascending=False)
'''
I did my data this way because one host can have multiple properties, 
and generally when I think of who is actively very busy I am looking 
for their monthly activity. Someone with many reviews per month will 
have a high customer traffic. below are the top 5, by ID, and name.
'''

''' % availability not vacant would be a best metric to determine, can we find this?'''

'''Decided to use a sum by, since the original field is an average of monthly reviews per property, but some hosts have multiple properties
these hosts also can have multiple low review properties but still be very busy but not show up in top 5 if using mean accross all properties.
'''

display(ab_sum[['host_id','host_name','reviews_per_month']].head(5))


,host_id,host_name,reviews_per_month
34629,219517861,Sonder (NYC),397.56
35807,244361589,Row NYC,111.72
35389,232251881,Lakshmee,80.63
18626,26432133,Danielle,68.02
31055,137274917,David,62.89


In [14]:
# How many neighborhood groups are available and which shows up the most?
print(f"There are {len(ab_data['neighbourhood_group'].unique())} unique neighborhood groups, they are: \n {ab_data['neighbourhood_group'].unique()}")
''''''
ab_data.groupby('neighbourhood_group',as_index=False).count().sort_values('id',ascending=False)[['neighbourhood_group','id']]


#48895

There are 5 unique neighborhood groups, they are: 
 ['Brooklyn' 'Manhattan' 'Queens' 'Staten Island' 'Bronx']


,neighbourhood_group,id
2,Manhattan,21661
1,Brooklyn,20104
3,Queens,5666
0,Bronx,1091
4,Staten Island,373


In [15]:
# Are private rooms the most popular in manhattan?
x = ab_data[["neighbourhood_group", "room_type",'reviews_per_month']].groupby(["neighbourhood_group", 'room_type'],as_index=False).sum(True).sort_values('reviews_per_month',ascending =False)

x[x["neighbourhood_group"] == "Manhattan"]

#again we are assuming reviews per month are the best metric for activity and popularity due to a lack of confirmed bookings per month record not existing and
#and under the assumption that guests are reviewinging immediately after their stay
#it  appear that the most popular neighborhood group was in fact "Entire home/apt"



,neighbourhood_group,room_type,reviews_per_month
6,Manhattan,Entire home/apt,11378.43
7,Manhattan,Private room,9141.42
8,Manhattan,Shared room,638.23


In [16]:
# Which hosts are the busiest and based on their reviews?
ab_data.groupby(['host_id','host_name'], as_index=False).sum(True).sort_values('number_of_reviews', ascending=False)[['host_id','host_name','number_of_reviews']].head()

#we decided to do of all time since we have the monthly metrics


,host_id,host_name,number_of_reviews
21304,37312959,Maya,2273
1052,344035,Brooklyn& Breakfast -Len-,2205
18626,26432133,Danielle,2017
20872,35524316,Yasu & Akiko,1971
21921,40176101,Brady,1818


In [17]:
#Which neighorhood group has the highest average price?
ab_data.groupby('neighbourhood_group',as_index=False).mean(True).round(2).sort_values('price',ascending=False)[['neighbourhood_group','price']]
#did the average price of the units by neighborhood group.



,neighbourhood_group,price
2,Manhattan,196.88
1,Brooklyn,124.38
4,Staten Island,114.81
3,Queens,99.52
0,Bronx,87.50


In [18]:
# Which neighbor hood group has the highest total price?
ab_data.groupby('neighbourhood_group',as_index=False).sum(True).round(2).sort_values('price',ascending=False)[['neighbourhood_group','price']]
#we decided on sum due the total price in the question

,neighbourhood_group,price
2,Manhattan,4264527
1,Brooklyn,2500600
3,Queens,563867
0,Bronx,95459
4,Staten Island,42825


In [19]:
#Which top 5 hosts have the highest total price?
#resorting old Sum data due to same group by condfitins
display(ab_sum[['host_id','host_name','price']].sort_values('price',ascending=False).head(5))

,host_id,host_name,price
34629,219517861,Sonder (NYC),82795
29393,107434423,Blueground,70331
32054,156158778,Sally,37097
34034,205031545,Red Awning,35294
19564,30283594,Kara,33581


In [25]:
# Who currently has no (zero) availability with a review count of 100 or more?

mask = (ab_data['number_of_reviews']>=100) & (ab_data['availability_365']==0)
display(ab_data[mask].groupby(['host_id','host_name'], as_index=False).count().sort_values('id', ascending=False)[['host_id','host_name','number_of_reviews']])

#I decided to count how many existing properties met these conditions.



,host_id,host_name,number_of_reviews
90,22959695,Gurpreet Singh,4
135,99392252,Michael,4
112,37818581,Sofia,2
131,94100043,Mohamed,2
118,44924546,Hayes,2
...,...,...,...
50,4434798,Bernard,1
51,4983320,Terri,1
52,5291029,Qiyao,1
53,5464042,Nicole,1


In [21]:
# What host has the highest total of prices and where are they located?
ab_data.groupby(['host_id','host_name','neighbourhood'], as_index=False).sum(True).sort_values('price', ascending=False)[['host_id','host_name','neighbourhood','price']].head(1)

#disgression


,host_id,host_name,neighbourhood,price
35431,219517861,Sonder (NYC),Financial District,57738


In [22]:
# When did Danielle from Queens last receive a review?
#use of max function
lr_dan = ab_data[(ab_data['host_name']== "Danielle")&(ab_data['neighbourhood_group']=='Queens')&(ab_data['last_review'].isnull() == False)][['host_id','host_name','neighbourhood_group','last_review']]
lr_dan.groupby(['host_id','host_name','neighbourhood_group']).max(False).sort_values('last_review',ascending=False)



,,,last_review
host_id,host_name,neighbourhood_group,
26432133,Danielle,Queens,2019-07-08
201647469,Danielle,Queens,2019-06-20
154256662,Danielle,Queens,2018-01-02


## Further Questions

1. Which host has the most listings?

In [27]:
'''
I had to look this up:
calculated_host_listings_count
It basically tells us the number of times that particular host has used airbnb in that dataset.
So if calculated_host_listings_count is 6 then u can see that host_name has exactly 6 rows in that dataset.'''
ab_data[['host_id','host_name','calculated_host_listings_count']].sort_values('calculated_host_listings_count',ascending=False).head(1)

,host_id,host_name,calculated_host_listings_count
39773,219517861,Sonder (NYC),327


2. How many listings have completely open availability?

In [28]:
list_data= ab_data[ab_data["availability_365"] == 365]
list_data['id'].count()

1295

3. What room_types have the highest review numbers?

In [30]:
ab_data[["neighbourhood_group", "room_type",'number_of_reviews']].groupby([ 'room_type'],as_index=False).sum(True).sort_values('number_of_reviews',ascending =False)

,room_type,number_of_reviews
0,Entire home/apt,580403
1,Private room,538346
2,Shared room,19256


# Final Conclusion

In this cell, write your final conclusion for each of the questions asked.

Also, if you uncovered some more details that were not asked above, please discribe them here.

-- Add your conclusion --